# sdmlib

`sdmlib` is a simple Python library providing an implementation of SDM that is intended for use with `numpy` arrays. I wrote this library for personal use while writing my undergraduate thesis in Neuroscience.

# Overview

Sparse Distributed Memory (SDM) is a technique for simulating computer memory with significantly more storage than is actually possible. Modern computer use $64$-bit memory, but SDM allows us to simulate $1000$-bit memory (or more!) by distributing the operations of reading and writing over a small subset of randomly sampled addresses. 

# Computer Memory
You can locate a house on a street using, for example, a $3$-digit combination of the digits $0$ through $9$. This gives you $10^3 = 1000$ possible addresses. Maybe your friend Robin lives at address $831$.

Similarly, you can specify a location in *computer memory* using a 64-digit combination of the binary digits ("bits") $0$ and $1$. This gives you $2^{64} = 18,446,744,073,709,551,616$ possible memory addresses to choose from. (In reality we do not use all of these addresses, but the principle still holds). We use these addresses to store data. Like addresses, data are stored in the form of $0$s and $1$s, and are usually $64$ bits long (just like street addresses). 

# Motivation
SDM was originally developed by Pentti Kanerva as a mathematical model for human long-term memory. Kanerva noted that high-dimensional spaces have some properties in common with human long-term memory. For example, similar memories can be 'nearby' in the same sense that points in high-dimensional space can be 'nearby'. Furthermore, two random memories tend to be unrelated just like two random points in high-dimensional space tend to be far apart. For Kanerva, a sufficiently high number of dimensions begins at least in the hundreds, if not in the thousands.

The simplest high-dimensional space is the binary space $\{0, 1 \}^N$. This space has $N$ dimensions, where each dimension can *only* take on the value $0$ or $1$. On a $64$-bit computer, every possible address is in $\{0, 1 \}^{64}$.

This space is a good candidate for our mathematical model of human long-term memory because:
1. it is high-dimensional
2. it is "simple" (only two values)
3. it can hypothetically be implemented on a digital computer

# The Problem
The possibility of implementing a model of human memory on a computer is exciting. However, even modern computers that use $64$-bit addresses are not sufficiently 'high-dimensional' for the properties Kanerva was interested in. We are interested in having, for example, $1000$ bit memory addresses. Sadly, this would require $2^{1000}$ locations to store data in our computer and our universe only have $2^{265}$ atoms, so we will run out of atoms before we can construct such a computer.

# Solution to the Problem
To solve this problem, Kanerva suggests only implementing a small, randomly sampled number of *hard addresses* - actual locations in computer memory corresponding to a point in high-dimensional space. We use $M$ to denote the number of hard addresses to implement. $M$ is often small, like $10^6$ (small relative to $2^N$). 

If we randomly pick an address from $\{ 0, 1 \}^N$, we are extremely unlikely to pick an address that has a corresponding hard address. In fact, the odds are

$$
\frac{M}{2^N}
$$

which, if we take $M=10^6$ and $N=1000$ (common values used in SDM), the odds are approximately $9.3 \times 10^{-296}$.

Let's arrange our hard addresses in an $M \times N$ matrix where each row is a hard address. We can call this matrix $A$.

$$
A = \begin{bmatrix}
1 & 0 & 0 & 1 & 1 \dots & 1 \\
0 & 0 & 1 & 1 & 0 \dots & 1 \\
1 & 1 & 0 & 1 & 1 \dots & 1 \\
1 & 0 & 0 & 0 & 0 \dots & 1 \\
\vdots & & & & & \vdots \\
0 & 1 & 1 & 1 & 0 \dots & 1 \\
\end{bmatrix}
$$

Imagine trying to write to an $N$-bit address called $x$. It is unlikely that $x$ is a row of $A$. How, then, can we write to this address? We instead pick hard addresses that are 'close' to $x$. In this case, two addresses are close if the *Hamming distance* between them is below some threshold $d$.

The Hamming distance between two binary strings of equal length is the number of places where their bits do not match. For example, the Hamming distance between $1001 \; 1111$ and $1011 \; 1110$ is $2$.

We go through each row $i$ in $A$. If the Hamming distance between $x$ $A_i$ is smaller than $d$, then we consider that address to be 'nearby'.

We can make a vector $h$ of length $M$ to keep track of the Hamming distance between each row of $A$ and our desired address $x$. We can make a similar vector $y$ that is $1$ where $h < d$ and $0$ otherwise.

<img src="{{site.url}}/assets/sdm1.drawio.svg" alt="">

To recap: we have a matrix $A$ of addresses. It has $M$ rows and $N$ columns, corresponding to the $M$ hard addresses and the $N$ bits of each address. Given an $N$-bit address $x$ that we would like to access, we construct a vector $y$. If a row $i$ of $A$ has a Hamming distance to $x$ that is smaller than $d$, we set $y_i$ to $1$ and otherwise set it to $0$. 

How do we write to memory? Let's say we have some data $w$ that we want to write. $w$ is a binary vector of length $U$. In normal computer memory, we pick a single address $x$ and overwrite the contents of that memory with $w$. However, in our new formulation, we have *multiple* addresses in $A$ that are considered nearby. We do not want to simply replace the contents of all of the nearby addresses with $w$. Why is this a bad idea? Because it's possible that some of those addresses are close to $x$ but also close to some other address $x'$, although $x$ and $x'$ may not themselves be close. We do not want to destroy information that was stored at previous times by just overwriting it.

Instead, we have a memory matrix $C$ of shape $M \times U$ that stores *counters*. For each element in $y_i$ of $y$, if the $y_i = 1$, then we will write $C_i$. For each bit $j$ from $1$ to $U$, if $w_j=1$, then we increment the $c_{ij}$. If $w_j$ is a $0$, then we decrement $C_{ij}$. The process of writing to memory becomes **distributed** across multiple addresses.